In [1]:
import sys
sys.path.append("..")
from pathlib import Path
import scraper
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib import parse

In [ ]:
data_dir = Path("../data")
format_name = "T20I"
format_dir = data_dir / format_name


In [2]:
# sanity check
comms = scraper.scrape_game(660011, 660235, debug=True)
comms

Getting commentary for series 660011 match 660235
Series: 660011 Match: 660235 innings: 1
Series: 660011 Match: 660235 innings: 2


[[{'_uid': 9153111,
   'id': 9153111,
   'inningNumber': 1,
   'ballsActual': None,
   'ballsUnique': None,
   'oversUnique': 0.01,
   'oversActual': 0.1,
   'overNumber': 1,
   'ballNumber': 1,
   'totalRuns': 1,
   'batsmanRuns': 0,
   'isFour': False,
   'isSix': False,
   'isWicket': False,
   'dismissalType': None,
   'byes': 0,
   'legbyes': 1,
   'wides': 0,
   'noballs': 0,
   'penalties': 0,
   'wagonX': 0,
   'wagonY': 0,
   'wagonZone': 0,
   'timestamp': None,
   'modified': '2016-10-06T18:06:41.000Z',
   'batsmanPlayerId': 47403,
   'bowlerPlayerId': 53755,
   'outPlayerId': None,
   'totalInningRuns': 1,
   'totalInningWickets': 0,
   'predictions': None,
   'title': 'Shapoor to Porterfield',
   'dismissalText': None,
   'commentPreTextItems': [{'type': 'HTML',
     'html': 'Hello all and welcome to the coverage of the final between Afghanistan and Ireland.'},
    {'type': 'HTML',
     'html': 'The news from the middle is that <b> Ireland</b> have won the toss and they ch

In [5]:
for year_path in format_dir.iterdir():
    with ThreadPoolExecutor() as pool:
        futures=[]
        for series_path in year_path.iterdir():
            matches_dir = series_path / "matches"

            if matches_dir.exists():
                for match_path in matches_dir.iterdir():
                    if (match_path / "meta.json").exists() and not (match_path / "ball_by_ball.json").exists():
                        match_meta = scraper.load_json(match_path / "meta.json")
                        match_id = match_meta["match"]["objectId"]
                        series_id = match_meta["match"]["series"]["objectId"]
                        if match_meta["match"]["coverage"] == "Y" and match_meta["match"]["status"] not in ["ABANDONED", "CANCELLED", "POSTPONED"]:
                            futures.append(pool.submit(scraper.scrape_and_save, scraper.scrape_game, match_path / "ball_by_ball.json", False, series_id, match_id, debug=False))
                            print(match_path)
                        else:
                            print("no comms available for {}".format(match_path))
                            
                            
            scraper.think_about_it_take_a_second(3)

        # wait for everything from year to finish to avoid overloading
        [r.result() for r in as_completed(futures)]
                
                    

../data/T20I/2013/icc-mens-t20-wc-qlf-8040/matches/afghanistan-vs-kenya-56th-match-group-b-660203
../data/T20I/2013/icc-mens-t20-wc-qlf-8040/matches/denmark-vs-nepal-3rd-match-group-b-660097
../data/T20I/2013/icc-mens-t20-wc-qlf-8040/matches/united-arab-emirates-vs-uganda-2nd-match-group-a-660095
no comms available for ../data/T20I/2013/icc-mens-t20-wc-qlf-8040/matches/bermuda-vs-uganda-57th-match-13th-place-play-off-660205
no comms available for ../data/T20I/2013/icc-mens-t20-wc-qlf-8040/matches/denmark-vs-papua-new-guinea-42nd-match-group-b-660175
../data/T20I/2013/icc-mens-t20-wc-qlf-8040/matches/italy-vs-uganda-21st-match-group-a-660133
../data/T20I/2013/icc-mens-t20-wc-qlf-8040/matches/canada-vs-united-states-of-america-5th-match-group-a-660101
../data/T20I/2013/icc-mens-t20-wc-qlf-8040/matches/united-arab-emirates-vs-ireland-18th-match-group-a-660127
no comms available for ../data/T20I/2013/icc-mens-t20-wc-qlf-8040/matches/denmark-vs-united-states-of-america-58th-match-15th-place